In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn import preprocessing

"""Change to the data folder"""
new_path = "../new_train/new_train/"
val_path = "../new_val_in/new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
#         data['p_in']-= numpy.mean(data['p_in'])
#         if self.transform:
#             min_max_scaler= preprocessing.MinMaxScaler()
#             for i in range(60):
#                 data['p_in'][i] = min_max_scaler.fit_transform(data['p_in'][i])
#                 data['v_in'][i] = min_max_scaler.fit_transform(data['v_in'][i])

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path,transform=True)
val_dataset = ArgoverseDataset(data_path=val_path,transform=True)
#print((val_dataset[0]))
#print(len(train_dataset[0]))

### Create a loader to enable batch processing

In [3]:
batch_sz = 1

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
#     if(len(str(maxVal)) > 3):
#         print(len(maxVal))
#     print("maxStringLength",len(str(maxVal)))
#     print("k   ",len(maxVal))
    
    inp = []
    out = []
    numbRows = 1000
    for scene in batch:
#         print(scene['p_in'])
        lanes = numpy.zeros((numbRows,19,2))
        lane_norm = numpy.zeros((numbRows,19,2))
        pIn = vIn = numpy.zeros((numbRows,19,2))
#         lane_norm =[0,0]
        lengthLane = min(numbRows,len(scene['lane']))
        pIn[:len(scene['p_in']),:,:2] = scene['p_in']
        vIn[:len(scene['v_in']),:,:2] = scene['v_in']
        lanes[:lengthLane,0,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,0,:2] = scene['lane_norm'] [:lengthLane,:2]
        inp.append(numpy.dstack([pIn,vIn,lanes,lane_norm]))
        out.append(numpy.dstack([scene['p_out'], scene['v_out']]))
        
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=7)

In [15]:

def val_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = []
    numbRows = 1000
    for scene in batch:
#         print(scene['p_in'])
        lanes = numpy.zeros((numbRows,19,2))
        lane_norm = numpy.zeros((numbRows,19,2))
        pIn = vIn = numpy.zeros((numbRows,19,2))
        lengthLane = min(numbRows,len(scene['lane']))
        pIn[:len(scene['p_in']),:,:2] = scene['p_in']
        vIn[:len(scene['v_in']),:,:2] = scene['v_in']
        lanes[:lengthLane,0,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,0,:2] = scene['lane_norm'] [:lengthLane,:2]
        inp.append(numpy.dstack([pIn,vIn,lanes,lane_norm]))
        scene_idx = np.array([scene['scene_idx'] for scene in batch])
        agent_id = np.array([scene['agent_id'] for scene in batch], dtype=object)
        track_id = np.stack([scene['track_id'][:,0] for scene in batch])
        
    inp = torch.FloatTensor(inp)
    return [inp, scene_idx, agent_id, track_id]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=val_collate, num_workers=4)

### Model Definition and Training

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()
        
#         # Number of hidden dimensions
#         self.hidden_dim = hidden_dim
        
#         # Number of hidden layers
#         self.n_layers = n_layers
        
#         # RNN
#         self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, nonlinearity='relu')
#         #print(self.rnn)
        
#         # Readout layer
#         self.fc = nn.Linear(hidden_dim, output_size)
    
#     def forward(self, x):
#         batch_size = x.size(0)
        
#         hidden = self.init_hidden(batch_size)
#         #print(hidden.shape)
#         out, hidden = self.rnn(x, hidden)
#         out = out.contiguous().view(-1, self.hidden_dim)
#         out = self.fc(out)
        
#         return out, hidden

        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True,dropout=0.5)
        self.fc = nn.Linear(hidden_dim, output_size)
    def forward(self, x,previous):
        device = torch.device("cuda:0")
        x = x.to(device)
        h0 = 0
        c0 = 0
        if(previous == 1):
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim,device=device).requires_grad_()
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim,device=device).requires_grad_()
        else:
            hn,cn = previous
            h0 = hn
            c0 = cn
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
#         print("forward outshape",out.shape)
        return out,(hn,cn)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim,device=torch.device("cuda:0"))
        return hidden

In [6]:
from statistics import mean
import random
import numpy as np

torch.cuda.empty_cache()
agent_id = 0
learning_rate = 50
momentum = 0.1
device = torch.device("cuda:0")
input_dim = 8    # input dimension
hidden_dim = 8  # hidden layer dimension
layer_dim = 1    # number of hidden layers
output_dim = 4   # output dimension

n_epochs = 5
lr=0.01

# Define Loss, Optimizer
#model = RNNModel(input_dim, output_dim, hidden_dim, layer_dim).to(device)
model = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
# model.load_state_dict(torch.load('./sliding3epoch.pth'))
model = model.to(device)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,lr_decay=0.00000001)
model.eval()

    
model.train()

print("test")
newOut = torch.zeros((1000,30,4))
batch = []
trainLossOverTime = numpy.zeros((205942,5))
try:
    for i_epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
        #     print("test")
            inp, out = sample_batch


            optimizer.zero_grad()
            newOut[:60,:,:] = out[0]
            initHidden = 1
            scaled_loss = 0
        #     print(inp)
            for i in range(30):

                output,hidden = model(inp[0].float().cuda(),initHidden)

                initHidden = hidden
                hn,cn = initHidden

                x = inp[0,:60,:,:4]
                x = torch.roll(x,-1,dims=1)
                inp[0,:60,:19,:4] = x
                inp[0,:60,18,:4] = out[0,:60,i,:]
                loss = nn.MSELoss()
                loss = loss(output.cuda(),newOut[:,i,:].cuda())
                loss.backward(retain_graph=True)
                scaled_loss += loss.item()
                optimizer.step()
            batch.append(scaled_loss/30)
            trainLossOverTime[i_batch,i_epoch] = scaled_loss

            if i_batch % 100 == 0:
                print("Epoch#:", i_epoch, " batch#:",i_batch ," avg loss (past 100): ",mean(batch))
                batch = []
                
except KeyboardInterrupt:
    torch.save(model.state_dict(), './sliding3epoch.pth')


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


test
Epoch#: 0  batch#: 0  avg loss (past 100):  29587.2453125
Epoch#: 0  batch#: 100  avg loss (past 100):  5695.765822682699
Epoch#: 0  batch#: 200  avg loss (past 100):  8932.263372131347
Epoch#: 0  batch#: 300  avg loss (past 100):  8091.572096872966
Epoch#: 0  batch#: 400  avg loss (past 100):  12413.940101476033
Epoch#: 0  batch#: 500  avg loss (past 100):  22483.81013597107
Epoch#: 0  batch#: 600  avg loss (past 100):  14709.850965820313
Epoch#: 0  batch#: 700  avg loss (past 100):  14821.918881103516
Epoch#: 0  batch#: 800  avg loss (past 100):  12709.192254923502
Epoch#: 0  batch#: 900  avg loss (past 100):  10898.28722302246
Epoch#: 0  batch#: 1000  avg loss (past 100):  9389.353409413656
Epoch#: 0  batch#: 1100  avg loss (past 100):  8539.028414805094


### Save and load Model

In [50]:
torch.save(model.state_dict(), './sliding3epoch.pth')

In [19]:
device = "cuda:0"
testmodel = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
testmodel.load_state_dict(torch.load('sliding3epoch.pth'))
testmodel = testModel.to(device)
testmodel.eval()

RNNModel(
  (lstm): LSTM(8, 8, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=8, out_features=4, bias=True)
)

## Test Submission

In [20]:
import numpy as np

save_file = "submissionsliding3.csv"

header = ["ID"]
header += ["v"+str(x) for x in range(1, 61)]
print(header)

with open(save_file, 'w') as f:
    f.write(",".join(header)+"\n")

device = "cuda:0"

# input_dim = 8    # input dimension
# hidden_dim = 8  # hidden layer dimension
# layer_dim = 1     # number of hidden layers
# output_dim = 4   # output dimension

# testmodel = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
# testmodel.load_state_dict(torch.load('sliding3epoch.pth'))
# testmodel.to(device)
# testmodel.eval()


for i_batch, sample_batch in enumerate(val_loader):
    if i_batch % 10 == 0:
        print("batch #: ", i_batch)
    inp, scene_idx, agent_ids, track_ids = sample_batch

    bool_loc = np.stack([np.squeeze(track_id) == np.repeat(ag_id,60) for ag_id, track_id in zip(agent_ids, track_ids)])


    initHidden = 1
    full_out = []
    for i in range(30):

        output,hidden = testmodel(inp[0].float().to(device),initHidden)
        initHidden = hidden
        hn,cn = initHidden
        full_out.append(output)
        x = inp[0,:60,:,:4]

        x = torch.roll(x,-1,dims=1)
        inp[0,:60, :19, :4] = x
        inp[0,:60,18,:4] = output[:60,:]
        inp = inp.to(device)

    out = torch.stack(full_out)
    
    
    out = out.transpose(0,1)
    out = out[:60, :, :2]
    out = out.reshape(60, -1)
    out = out[bool_loc]
    with open(save_file, "a") as f:
        for row, scene_num in zip(out.detach().cpu().numpy().astype(int).astype(str), scene_idx):
            f.write(str(scene_num)+","+",".join(row)+"\n")


['ID', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60']
batch #:  0
batch #:  10
batch #:  20
batch #:  30
batch #:  40
batch #:  50
batch #:  60
batch #:  70
batch #:  80
batch #:  90
batch #:  100
batch #:  110
batch #:  120
batch #:  130
batch #:  140
batch #:  150
batch #:  160
batch #:  170
batch #:  180
batch #:  190
batch #:  200
batch #:  210
batch #:  220
batch #:  230
batch #:  240
batch #:  250
batch #:  260
batch #:  270
batch #:  280
batch #:  290
batch #:  300
batch #:  310
batch #:  320
batch #:  330
batch #:  340
batch #:  350
batch #:  360
batch #:  370
batch #:  380
batch #:  390
batch #:  400
batch #:  410
batch

### Visualization funtions

In [55]:
def make_a_histogram(sample_batch, agent_id, xPos, yPos, xVel, yVel):
    inp, out = sample_batch
    batch_sz = inp.size(0)
    #agent_sz = inp.size(1)
    
    for i in range(batch_sz):
        #hist_data_xPos = np.zeros((60,19));
        #hist_data_yPos = np.zeros((60,19));
        #hist_data_xVel = np.zeros((60,19));
        hist_data_yVel = np.zeros((60,19));
        
        for j in range(60):
            #hist_data_xPos[j] = (inp[i, j,:,0])
            #hist_data_yPos[j] = (inp[i, j,:,1])
            #hist_data_xVel[j] = (inp[i, j,:,2])
            hist_data_yVel[j] = (inp[i, j,:,3])
            
        for j in range(len(hist_data_yVel)):
            for k in range(len(hist_data_yVel[j])):
                #xPos.append(hist_data_xPos[j][k])
                #yPos.append(hist_data_yPos[j][k])
                #xVel.append(hist_data_xVel[j][k])
                yVel.append(hist_data_yVel[j][k])
    
    """
    hist_data_xPos = np.zeros((60,19));
    hist_data_yPos = np.zeros((60,19));
    hist_data_xVel = np.zeros((60,19));
    hist_data_yVel = np.zeros((60,19));
    
    for i in range(60):
        hist_data_xPos[i] = (inp[0, i,:,0])
        hist_data_yPos[i] = (inp[0, i,:,1])
        hist_data_xVel[i] = (inp[0, i,:,2])
        hist_data_yVel[i] = (inp[0, i,:,3])
    
    xPos = np.zeros(60*19)
    for i in range(len(hist_data_xPos)):
        for j in range(len(hist_data_xPos[i])):
            xPos[i*19+j] = hist_data_xPos[i][j]
    
    #hist_data_xPos = hist_data_xPos.flatten()
    hist_data_yVel = hist_data_yPos.flatten()
    hist_data_xPos = hist_data_xVel.flatten()
    hist_data_yVel = hist_data_yVel.flatten()
    #print(xPos)
    
    n,bins,patches = plt.hist(x=xPos,bins='auto',alpha=0.7,rwidth=0.85)
    plt.grid(axis='y',alpha=0.75)
    maxfreq = n.max()
    plt.ylim(ymax=np.ceil(maxfreq/10) * 10 if maxfreq % 10 else maxfreq + 10)
    """

In [48]:
def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])